In [2]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

In [17]:
df = pd.read_csv("Forcing.txt", delimiter="	", names=["year","month","day","P","Q","EP"])
df.index = df.apply(lambda x: pd.Timestamp(f'{int(x.year)}-{int(x.month)}-{int(x.day)}'),axis=1)
df.drop(columns=["year","month","day"],inplace=True)
df.head()

,P,Q,EP
1997-08-01,8.888,0.388,1.715079
1997-08-02,0.265,0.517,3.779446
1997-08-03,1.034,0.543,2.157360
1997-08-04,0.000,0.448,4.788141
1997-08-05,0.000,0.428,4.964434


In [21]:
# specifiy number of particles
n_particles = 10

In [43]:
# set up storage terms 
# allocate Si, Su, Sf, Ss, Eidt, Eadt, Qtotdt
t_max = len(df)

storage_memory = []
# could be more ellegant with numpy but lets just start with 10 lists of 10 arrays
for i in range(n_particles):
    Si        = np.zeros(t_max) # Interception storage
    Su        = np.zeros(t_max) # Unsaturated Rootzone Storage
    Sf        = np.zeros(t_max) # Fastflow storage
    Ss        = np.zeros(t_max) # Groundwater storage
    Ei_dt     = np.zeros(t_max) # interception evaporation
    Ea_dt     = np.zeros(t_max) # actual evaportation
    Qs_dt_lst = np.zeros(t_max) # list of slow flow 
    Qf_dt_lst = np.zeros(t_max) # list of fast flow
    Q_tot_dt  = np.zeros(t_max) # total flow
    storage_memory.append([Si, Su, Sf, Ss, Ei_dt, Ea_dt, Qs_dt_lst, Qf_dt_lst, Q_tot_dt])


In [44]:
## Array of parameters min/max bounds
##                      Imax,  Ce,  Sumax, beta,  Pmax,  Tlag,   Kf,   Ks
p_min_initial= np.array([0,   0.2,  40,    .5,   .001,   0,     .01,  .0001])
p_max_initial = np.array([8,    1,  800,   4,    .3,     10,    .1,   .01])

## Array of initial storage terms - we keep these constant for now 
##              Si,  Su, Sf, Ss
s_in = np.array([0,  100,  0,  5  ])

Note notation 3 is used with p for parameters <br>
<img src="figures\Lecture1A_slide26_notation.png" alt="Lecture1A_slide26_notation" style="height: 300px"/>

In [45]:
# for initial run sample a set of parameters - start with simpel linear random:  min+[0,1] * (max-min) -> easily changes after
array_random_num = np.array([[np.random.random() for i in range(len(p_max_initial))] for i in range(n_particles)])
p_intial = p_min_initial + array_random_num * (p_max_initial-p_min_initial)

In [46]:
# inital value array for every particle
p_intial[0],p_intial[1]

(array([8.82708734e-01, 8.52714659e-01, 6.43734946e+02, 5.56118365e-01,
        1.12354901e-01, 5.38085627e+00, 4.66218577e-02, 5.80197002e-03]),
 array([3.04103955e+00, 4.58651758e-01, 2.02355814e+02, 3.53228521e+00,
        2.35809550e-01, 1.18171400e+00, 7.22928520e-02, 7.83227715e-03]))

In [53]:
particle_n = 0
step_n = 0
HBVMod_forward(p_intial[0], df, s_in, storage_memory[particle_n], step_n)

(array([0.88270873, 0.88270873, 0.        , ..., 0.        , 0.        ,
        0.        ]),
 array([104.81631088, 104.81631088,   0.        , ...,   0.        ,
          0.        ,   0.        ]),
 array([2.70960177, 2.70960177, 0.        , ..., 0.        , 0.        ,
        0.        ]),
 array([4.9891814, 4.9891814, 0.       , ..., 0.       , 0.       ,
        0.       ]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.32857695, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]),
 array([0.02911601, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]),
 array([0.13250426, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]),
 array([0.16162027, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]))

In [52]:
def HBVMod_forward(par, df, s_in, memory_terms, step_n):
    """
    Function to run the forward part of one timestep of the HBV model     
    parameters
    ----------
    par: array_like
        array/list containing 8 parameters: Imax,  Ce,  Sumax, beta,  Pmax,  Tlag,   Kf,   Ks (floats)
    df: pandas.core.frame.DataFrame
        DataFrame containing 'P', 'Q', 'EP' columns as forcing for the model.
    s_in: array_like
        array/list containing 4 storage terms which are input to the timestep: Si,  Su, Sf, Ss (floats)
    memory_terms: list of arrays
        list of arrays which store: Si, Su, Sf, Ss, Ei_dt, Ea_dt, Qs_dt_lst, Qf_dt_lst, Q_tot_dt
    step_n: int
        nth step which formard model takes: used to determin which Precipitaion & evaporation to use

    Returns
    ----------
    Obj, df: float, pandas.core.frame.DataFrame
        return of the objective function and corresponding dataframe

    """
    #HBVpareto Calculates values of 3 objective functions for HBV model
    
    # define parameters 
    I_max  = par[0]   # maximum interception
    Ce     = par[1]   # Ea = Su / (sumax * Ce) * Ep
    Su_max = par[2]   # ''
    beta   = par[3]   # Cr = (su/sumax)**beta
    P_max  = par[4]   # Qus = Pmax * (Su/Sumax)
    T_lag  = par[5]   # used in triangular transfer function
    Kf     = par[6]   # Qf=kf*sf
    Ks     = par[7]   # Qs=Ks*Ss
    
    # precipitation is given by the first column, Q0 by the second and  
    # Precipitaion & evaporation are inputs
    Prec   = df.P.iloc[step_n]
    Etp    = df.EP.iloc[step_n]
    
    # For now skip evap
    # Qo     = df.Q.values
    
    # initialize the storage coefficients Si, Su, Sf, Ss
    Si[0] = s_in[0]
    Su[0] = s_in[1]
    Sf[0] = s_in[2]
    Ss[0] = s_in[3]

    # time step of one day
    dt = 1

    i = step_n # maybe refactor later but useful for now
    
    
    P_dt  = Prec * dt
    Ep_dt = Etp  * dt

    # Interception Reservoir
    if P_dt > 0:
        # if there is rain, no evap
        Si[i]    = Si[i] + P_dt               # increase the storage
        Pe_dt    = max((Si[i] - I_max) / dt, 0)
        Si[i]    = Si[i] - Pe_dt 
        Ei_dt[i] = 0                          # if rainfall, evaporation = 0 as too moist 
    else:
        # Evaporation only when there is no rainfall
        Pe_dt    = 0                      # nothing flows in so must be 0
        Ei_dt[i] = min(Ep_dt, Si[i] / dt) # evaporation limited by storage
        Si[i]    = Si[i] - Ei_dt[i] 

    # if not the end, initialise the next storage
    if i < t_max - 1:
        Si[i+1] = Si[i]


    # split flow into Unsaturated Reservoir and Fast flow
    if Pe_dt > 0:
        cr     = (Su[i] / Su_max)**beta
        Qiu_dt = (1 - cr ) * Pe_dt      # flux from Ir to Ur
        Su[i]  = Su[i] + Qiu_dt
        Quf_dt = cr  * Pe_dt            # flux from Su to Sf
    else:
        Quf_dt=0

    # Transpiration
    Ep_dt    = max(0, Ep_dt - Ei_dt[i])        # Transpiration 
    Ea_dt[i] = Ep_dt  * (Su[i] / (Su_max * Ce))
    Ea_dt[i] = min(Su[i], Ea_dt[i])            # limited by water in soil 
    Su[i]    = Su[i] - Ea_dt[i]

    # Percolation
    Qus_dt = P_max * (Su[i] / Su_max) * dt # Flux from Su to Ss
    Su[i]  = Su[i] - Qus_dt

    # make the final steps to update next step
    if i < t_max - 1:
        Su[i+1]=Su[i]

    # Fast Reservoir
    Sf[i] = Sf[i] + Quf_dt
    Qf_dt = dt * Kf * Sf[i]
    Sf[i] = Sf[i] - Qf_dt
    if i < t_max -  1:
        Sf[i+1] = Sf[i]

    # Slow Reservoir
    Ss[i] = Ss[i] + Qus_dt
    Qs_dt = Ss[i] * Ks * dt
    Ss[i] = Ss[i] - Qs_dt
    if i < t_max-1:
        Ss[i+1] = Ss[i]

    Q_tot_dt[i] = Qs_dt + Qf_dt
    Qs_dt_lst[i] = Qs_dt
    Qf_dt_lst[i] = Qf_dt
       
    memory_terms = Si, Su, Sf, Ss, Ei_dt, Ea_dt, Qs_dt_lst, Qf_dt_lst, Q_tot_dt
    return memory_terms

In [54]:
# # 'normally' post processing steps after whole run 

# # Check Water Balance
# Sf  = Si[-1] + Ss[-1] + Sf[-1] + Su[-1] # final storage
# s_in = sum(s_in)                          # initial storage
# WB  = sum(Prec) - sum(Ei_dt)- sum(Ea_dt) - sum(Q_tot_dt) + s_in - Sf


# # Offset Q
# Weigths = Weigfun(T_lag)

# Qm = np.convolve(Q_tot_dt, Weigths)
# Qm = Qm[0:t_max]

# Qs_dt_lst = np.convolve(Qs_dt_lst, Weigths)
# Qs_dt_lst = Qs_dt_lst[0:t_max]

# Qf_dt_lst = np.convolve(Qf_dt_lst, Weigths)
# Qf_dt_lst = Qf_dt_lst[0:t_max]